### Necessary packages

In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from textblob import Word
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier

### Dataset

In [2]:
data = pd.read_csv("dataset/news.csv")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6335 entries, 0 to 6334
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  6335 non-null   int64 
 1   title       6335 non-null   object
 2   text        6335 non-null   object
 3   label       6335 non-null   object
dtypes: int64(1), object(3)
memory usage: 198.1+ KB


In [3]:
data.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


### TfidfVectorizer

#### Cleaning text

In [4]:
# Stopwords
file = open('dataset\list_stopwords.txt')
list_stopwords = file.read()
#list_stopwords.split(",")

nltk.download('stopwords')
stop_words=stopwords.words('english')+ list_stopwords.split(",")
stop_words=np.unique(stop_words).tolist()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\NADIA\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
data['text']

0       Daniel Greenfield, a Shillman Journalism Fello...
1       Google Pinterest Digg Linkedin Reddit Stumbleu...
2       U.S. Secretary of State John F. Kerry said Mon...
3       — Kaydee King (@KaydeeKing) November 9, 2016 T...
4       It's primary day in New York and front-runners...
                              ...                        
6330    The State Department told the Republican Natio...
6331    The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...
6332     Anti-Trump Protesters Are Tools of the Oligar...
6333    ADDIS ABABA, Ethiopia —President Obama convene...
6334    Jeb Bush Is Suddenly Attacking Trump. Here's W...
Name: text, Length: 6335, dtype: object

In [6]:
# Cleaning
data['text'] = data['text'].apply(lambda x:" ".join(x.lower() for x in x.split()))
data['text'] = data['text'].str.replace('[^\w\s]', ' ') # non alphanumeric characters
data['text'] = data['text'].apply(lambda x:' '.join(x for x in x.split() if x not in stop_words))
data['text'] = data['text'].apply(lambda x:' '.join([Word(x).lemmatize() for x in x.split(" ")]))
data['text'] = data['text'].str.replace('[\d]','') # numeric characters
data['text']

0       daniel greenfield shillman journalism fellow f...
1       google pinterest digg linkedin reddit stumbleu...
2       u secretary state john f kerry monday stop par...
3       kaydee king kaydeeking november   lesson tonig...
4       primary day new york front runner hillary clin...
                              ...                        
6330    state told republican national committee email...
6331    p pb stand plutocratic pentagon posted oct   w...
6332    anti trump protester tool oligarchy reform pro...
6333    addis ababa ethiopia president obama convened ...
6334    jeb bush suddenly attacking trump matter jeb b...
Name: text, Length: 6335, dtype: object

#### Spliting data

In [7]:
# labels
labels = data.label

# Split data
X_train, X_test, y_train, y_test = train_test_split(data['text'], 
                                                    labels, test_size = 0.2, random_state = 42)

#### TFidf

In [8]:
# TfidfVectorizer (train set)
vectorizer = TfidfVectorizer(max_features=1000, stop_words='english', max_df=0.7) #max_features=1000
vectors = vectorizer.fit_transform(X_train)
dense = vectors.todense()
vocabulary = vectorizer.get_feature_names()
denselist = dense.tolist()
df_train = pd.DataFrame(denselist, columns=vocabulary)
df_train.head(2)

,ability,able,abortion,access,according,account,accused,act,action,activist,...,world,worse,worth,written,wrong,wrote,year,yes,york,young
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.07912,0.0,0.0,0.0


In [9]:
# TfidfVectorizer (test set)
vectors_test = vectorizer.fit_transform(X_test)
dense_test = vectors_test.todense()
vocabulary_test = vectorizer.get_feature_names()
denselist_test = dense_test.tolist()
df_test = pd.DataFrame(denselist_test, columns=vocabulary_test)
df_test.head(2)

,ability,able,abortion,access,according,account,accused,act,action,activist,...,worse,worth,wrong,wrote,year,yemen,yes,york,young,zone
0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.026998,0.0,0.0,...,0.0,0.0,0.0,0.0,0.015280,0.0,0.0,0.000000,0.0,0.042816
1,0.0,0.0,0.0,0.0,0.091331,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.064604,0.0,0.0,0.072174,0.0,0.000000


### PassiveAgressiveClassifier

In [25]:
pac = PassiveAggressiveClassifier(max_iter = 100)
pac.fit(df_train, y_train)

PassiveAggressiveClassifier(C=1.0, average=False, class_weight=None,
                            early_stopping=False, fit_intercept=True,
                            loss='hinge', max_iter=100, n_iter_no_change=5,
                            n_jobs=None, random_state=None, shuffle=True,
                            tol=0.001, validation_fraction=0.1, verbose=0,
                            warm_start=False)

In [26]:
y_pred = pac.predict(df_test)
accuracy = accuracy_score(y_test, y_pred)
accuracy

0.6179952644041041

In [27]:
# confusion matrix
confusion_matrix(y_test,y_pred, labels=['FAKE','REAL'])

array([[400, 228],
       [256, 383]], dtype=int64)